In [20]:
import numpy as np
import pandas as pd

In [21]:
data = pd.read_csv("fairness_data.csv")

In [ ]:
data.head()

In [22]:
df = data.astype(int)

In [23]:
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income,Z,workclass_Federal-gov,workclass_Local-gov,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,...,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,1.897347e+05,10.118460,1101.430344,88.595418,40.938017,0.165141,0.499381,0.031091,0.068551,...,0.001371,0.003870,0.000442,0.002233,0.001216,0.000641,0.000575,0.913095,0.001835,0.000509
std,13.217870,1.056392e+05,2.552881,7506.430084,404.956092,12.007508,0.371312,0.500005,0.173566,0.252691,...,0.037002,0.062088,0.021026,0.047207,0.034854,0.025316,0.023971,0.281698,0.042803,0.022547
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.173882e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.783160e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,47.000000,2.379260e+05,13.000000,0.000000,0.000000,45.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



In [32]:
sensitive_attr = df['Z']
target = df['income']

# FTU means we exclude Z from the training features
X = df.drop(columns=['income', 'Z'])
y = target
z = sensitive_attr # We need Z separately for evaluation


# Split data (Standard 80/20 or similar split, ensure random_state for reproducibility)
# pass 'z' implies we need to track z for the test set.
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(
    X, y, z, test_size=0.2, random_state=42
)

# Train the Fairness Through Unawareness (FTU) Classifier
model = GradientBoostingClassifier(random_state=42)

model.fit(X_train, y_train)

# Calculate Test-Set Accuracy
y_pred_hard = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_hard)

print(f"Test-set Accuracy: {accuracy:.4f}")
if accuracy >= 0.83:
    print(">> Requirement met: Accuracy is >= 83%")
else:
    print(">> Warning: Accuracy is below 83%. Consider tuning hyperparameters.")


Test-set Accuracy: 0.8381
>> Requirement met: Accuracy is >= 83%


In [ ]:

# Calculate Fairness Statistics from Part (a)
# We need the predicted probabilities (sigma(f(x))) for the formulas
y_pred_prob = model.predict_proba(X_test)[:, 1]

# Helper function to calculate mean based on condition
def get_mean_pred(probs, condition_mask):
    return np.mean(probs[condition_mask])

def get_mean_error(y_true, probs, condition_mask):
    # Mean Absolute Error: |y - y_hat|
    residuals = np.abs(y_true - probs)
    return np.mean(residuals[condition_mask])

# --- DP: Demographic Parity ---
# E[sigma(f(x)) | Z=1] - E[sigma(f(x)) | Z=0]
dp_val = (get_mean_pred(y_pred_prob, z_test == 1) - 
          get_mean_pred(y_pred_prob, z_test == 0))

# --- EO: Equalized Odds (on Positive Class Y=1) ---
# E[sigma(f(x)) | Z=1, Y=1] - E[sigma(f(x)) | Z=0, Y=1]
mask_z1_y1 = (z_test == 1) & (y_test == 1)
mask_z0_y1 = (z_test == 0) & (y_test == 1)

eo_val = (get_mean_pred(y_pred_prob, mask_z1_y1) - 
          get_mean_pred(y_pred_prob, mask_z0_y1))

# --- PP: Predictive Parity (Error Parity) ---
# E[|Y - sigma(f(x))| | Z=1] - E[|Y - sigma(f(x))| | Z=0]
pp_val = (get_mean_error(y_test, y_pred_prob, z_test == 1) - 
          get_mean_error(y_test, y_pred_prob, z_test == 0))

print("-" * 30)
print(f"DP (Demographic Parity): {dp_val:.4f}")
print(f"EO (Equalized Odds):     {eo_val:.4f}")
print(f"PP (Predictive Parity):  {pp_val:.4f}")
print("-" * 30)

------------------------------
DP (Demographic Parity): -0.1697
EO (Equalized Odds):     -0.1911
PP (Predictive Parity):  -0.2439
------------------------------


# using all the features, including the sensitive attribute Z

In [34]:
z_train.values

array([0, 0, 0, ..., 0, 1, 0], shape=(36177,))

In [35]:
X_train.values

array([[    32, 282611,     13, ...,      1,      0,      0],
       [    45, 192323,     12, ...,      0,      0,      1],
       [    45, 144086,      7, ...,      1,      0,      0],
       ...,
       [    31, 157568,     10, ...,      1,      0,      0],
       [    37, 176900,      9, ...,      1,      0,      0],
       [    56,  51662,      7, ...,      1,      0,      0]],
      shape=(36177, 103))

In [37]:
X_train_c = X_train.copy()

In [38]:
X_train_c

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
7963,32,282611,13,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
26402,45,192323,12,0,0,66,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
31411,45,144086,7,0,0,50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13367,39,168355,9,0,0,70,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
38742,51,187686,9,0,0,38,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,26,34393,10,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
44732,24,542762,13,0,0,50,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
38158,31,157568,10,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
860,37,176900,9,0,0,99,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [39]:
z_train

7963     0
26402    0
31411    0
13367    1
38742    0
        ..
11284    1
44732    0
38158    0
860      1
15795    0
Name: Z, Length: 36177, dtype: int64

In [40]:
X_train_c['z'] = z_train.values

In [41]:
X_train_c

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,z
7963,32,282611,13,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
26402,45,192323,12,0,0,66,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
31411,45,144086,7,0,0,50,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
13367,39,168355,9,0,0,70,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
38742,51,187686,9,0,0,38,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,26,34393,10,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
44732,24,542762,13,0,0,50,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
38158,31,157568,10,0,0,40,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
860,37,176900,9,0,0,99,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [43]:
X_train_c = X_train.copy()
X_train_c['Z'] = z_train.values

X_test_c = X_test.copy()
X_test_c['Z'] = z_test.values

In [44]:
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train_c, y_train)

# 3. Define the Marginalized Prediction Function f_inv(x)
def predict_marginalized(model, X_data, sensitive_col):
    """
    Predicts by setting Z to 0 and Z to 1 for every row, 
    then averaging the resulting probabilities.
    """
    # Create copies where Z is forced to 0 and 1
    X_z0 = X_data.copy()
    X_z0[sensitive_col] = 0
    
    X_z1 = X_data.copy()
    X_z1[sensitive_col] = 1
    
    # Get probabilities for both counterfactuals
    # Note: we use predict_proba for the summation
    prob_z0 = model.predict_proba(X_z0)[:, 1]
    prob_z1 = model.predict_proba(X_z1)[:, 1]
    
    # Compute the average (marginalize out Z)
    # f_inv(x) = 1/|Z| * sum(f(x, z))
    avg_probs = (prob_z0 + prob_z1) / 2.0
    
    # Convert probabilities to hard predictions (0 or 1) using 0.5 threshold
    hard_preds = (avg_probs >= 0.5).astype(int)
    
    return hard_preds, avg_probs

# 4. Evaluate on Test Set
y_pred_inv, y_prob_inv = predict_marginalized(model, X_test_c, 'Z')

# --- Accuracy ---
acc_c = accuracy_score(y_test, y_pred_inv)
print(f"Part (c) Marginalized Accuracy: {acc_c:.4f}")
if acc_c >= 0.81:
    print(">> Requirement met: Accuracy is >= 81%")


Part (c) Marginalized Accuracy: 0.8286
>> Requirement met: Accuracy is >= 81%


In [ ]:

# --- Calculate Statistics (DP, EO, PP) using the new probabilities ---
# (Reusing helper functions from Part a)

# DP: E[f_inv | Z=1] - E[f_inv | Z=0]
dp_c = (get_mean_pred(y_prob_inv, z_test == 1) - 
        get_mean_pred(y_prob_inv, z_test == 0))

# EO: E[f_inv | Z=1, Y=1] - E[f_inv | Z=0, Y=1]
mask_z1_y1 = (z_test == 1) & (y_test == 1)
mask_z0_y1 = (z_test == 0) & (y_test == 1)
eo_c = (get_mean_pred(y_prob_inv, mask_z1_y1) - 
        get_mean_pred(y_prob_inv, mask_z0_y1))

# PP: E[|Y - f_inv| | Z=1] - E[|Y - f_inv| | Z=0]
pp_c = (get_mean_error(y_test, y_prob_inv, z_test == 1) - 
        get_mean_error(y_test, y_prob_inv, z_test == 0))

print("-" * 30)
print(f"Part (c) Statistics:")
print(f"DP: {dp_c:.4f}")
print(f"EO: {eo_c:.4f}")
print(f"PP: {pp_c:.4f}")
print("-" * 30)

------------------------------
Part (c) Statistics:
DP: -0.0038
EO: 0.0565
PP: -0.1551
------------------------------
Comparison to Part (b):
You should typically see that DP decreases (improves) compared to Part (b).
This is because we explicitly remove the dependence on Z by averaging it out,
isolating the signal that comes only from the non-sensitive features.
